## ⚠️ Important: Restart Kernel

If you've just updated the casased module, **restart the kernel** before running this notebook:
- VS Code: Click "Restart" button in notebook toolbar, or
- Jupyter menu: Kernel → Restart Kernel

This ensures you're using the latest version with browser automation support.

# <center>casased - Casablanca Stock Exchange Data Retriever</center>

This notebook demonstrates the features of the `casased` Python library for retrieving historical and intraday data from the Casablanca Stock Exchange (Bourse de Casablanca).

**Data Source:** [Medias24 API](https://medias24.com/bourse)

**✨ New in v0.1.5:** Automatic Cloudflare bypass using headless browser automation (nodriver/seleniumbase)

## Features
- 📈 Historical price data for stocks and indices (MASI, MSI20)
- ⏱️ Intraday price data
- 🏦 Multiple asset loading
- 📋 Session data, key indicators, dividends
- 📉 Index summaries and weights
- 🚀 Automatic 403 error bypass with browser automation

## Setup

For best results with Cloudflare protection, install browser automation tools:
```bash
pip install casased[cloudflare]
# or manually: pip install nodriver seleniumbase
```

In [49]:
# Restart kernel if you've just updated the module
# Jupyter menu: Kernel → Restart Kernel

# Install casased with Cloudflare bypass support (uncomment if needed)
# !pip install casased[cloudflare]

# Import casased functions
from casased import (
    notation,           # Asset name/ISIN mapping
    get_isin_by_name,   # Get ISIN by asset name
    get_history,        # Historical price data
    loadmany,           # Load multiple assets
    get_intraday,       # Intraday price data
    getCours,           # Current session data
    getKeyIndicators,   # Key indicators
    getDividend,        # Dividend history
    getIndex,           # Index information
)
import pandas as pd

print(f"casased v0.1.5 loaded - {len(notation())} assets available")
print("✓ Browser automation enabled for Cloudflare bypass")

casased v0.1.5 loaded - 81 assets available
✓ Browser automation enabled for Cloudflare bypass


## 1. Available Assets

Get the list of all available stock names and their ISIN codes.

In [43]:
# List all available assets
assets = notation()
print(f"Total assets available: {len(assets)}")
print(assets[:10])  # Show first 10

Total assets available: 81
['Addoha', 'AFMA', 'Afric Indus', 'Afriquia Gaz', 'Agma', 'Akdital', 'Alliances', 'Aluminium Maroc', 'Aradei Capital', 'ATLANTASANAD']


In [44]:
# Get ISIN code for a specific asset
isin = get_isin_by_name('BCP')
print(f"ISIN for BCP: {isin}")

ISIN for BCP: MA0000011884


## 2. Historical Price Data

Retrieve historical OHLCV data for any listed stock using the Medias24 API.

**Note:** If you see warnings about 403 errors, the library automatically uses browser automation (nodriver/seleniumbase) to bypass Cloudflare protection. This may take a few extra seconds.

In [47]:
# Load historical data for a single stock
# Using a shorter date range for faster execution
data = get_history('BCP', start='2024-06-01', end='2024-12-31')
print(f"Data shape: {data.shape}")
print(f"Columns: {list(data.columns)}")
print(f"Date range: {data.index.min()} to {data.index.max()}")
data.head(10)

Data shape: (142, 5)
Columns: ['Value', 'Min', 'Max', 'Variation', 'Volume']
Date range: 2024-06-03 to 2024-12-31


,Value,Min,Max,Variation,Volume
Date,,,,,
2024-06-03,294.00,294.00,299.50,0.00,17081
2024-06-04,290.30,290.30,294.00,-1.26,56859
2024-06-05,290.00,290.00,291.95,-0.10,58853
2024-06-06,293.00,290.00,298.00,1.03,1734452
2024-06-07,293.95,293.00,297.00,0.32,656054
2024-06-10,292.00,292.00,294.00,-0.66,260848
2024-06-11,291.95,291.95,292.00,-0.02,833
2024-06-12,291.00,290.05,292.00,-0.33,2137
2024-06-13,289.00,289.00,291.00,-0.69,25582


In [50]:
# Load MASI index history
masi = get_history('MASI')
print(f"MASI data shape: {masi.shape}")
masi.tail()

MASI data shape: (0, 0)


""


In [51]:
# Load MSI20 index history
msi20 = get_history('MSI20')
print(f"MSI20 data shape: {msi20.shape}")
msi20.tail()

MSI20 data shape: (0, 0)


""


## 3. Loading Multiple Assets

Load data for multiple stocks at once and compare.

**Note:** When loading multiple assets, browser automation may be triggered multiple times. This can take 10-30 seconds per asset if Cloudflare is blocking direct requests.

In [ ]:
# Load multiple assets - closing prices
# Using shorter date range and fewer assets for demonstration
print("Loading data for 2 banks (may take 30-60 seconds with browser automation)...")
banks = loadmany(['BCP', 'Attijariwafa'], 
                 start='2024-06-01', end='2024-06-30')
print(f"Shape: {banks.shape}")
print(f"Columns: {list(banks.columns)}")
banks.dropna().head()

Shape: (122, 3)


,BCP,Attijariwafa,BMCI
Date,,,


In [ ]:
# Load volumes for multiple assets
print("Loading volume data (may take 30-60 seconds)...")
volumes = loadmany(['BCP', 'Attijariwafa'], 
                   start='2024-06-01', end='2024-06-30', 
                   feature='Volume')
print(f"Shape: {volumes.shape}")
volumes.dropna().head()

,BCP,BMCI,BOA,CIH


## 4. Intraday Data

Get real-time intraday price data for stocks and indices.

**Note:** Intraday data availability depends on market hours. Browser automation will be used if needed.

In [ ]:
# Get intraday data for a stock (market hours only)
print("Fetching intraday data (may take 10-30 seconds)...")
intraday = get_intraday('BCP')
if not intraday.empty:
    print(f"✓ Intraday data retrieved - Shape: {intraday.shape}")
    print(f"Columns: {list(intraday.columns)}")
    display(intraday.head(10))
else:
    print("⚠ Intraday data not available (market may be closed)")

Status: 403
Content type: text/html; charset=UTF-8
Text sample: <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helve


In [ ]:
# Get MASI intraday
print("Fetching MASI intraday data...")
masi_intraday = get_intraday('MASI')
if not masi_intraday.empty and 'Value' in masi_intraday.columns:
    print(f"✓ MASI intraday shape: {masi_intraday.shape}")
    display(masi_intraday.head())
else:
    print("⚠ MASI intraday data not available")

MASI intraday shape: (1, 2)


,result,message
0,"{'labels': ['09:30', '09:31', '09:32', '09:33'...",200 OK


## 5. Session Data & Key Indicators

Get detailed session data, key indicators, and dividend history (from Bourse de Casablanca).

In [ ]:
# Get session data for a stock
cours = getCours('BOA')
print("Available keys:", list(cours.keys()))
cours.get('Données_Seance', {})

{}

In [ ]:
# Get key indicators
indicators = getKeyIndicators('Attijariwafa')
print("Available keys:", list(indicators.keys()))
indicators.get('Info_Societe', {})

Could not fetch index recap: HTTPSConnectionPool(host='www.casablanca-bourse.com', port=443): Max retries exceeded with url: /bourseweb/index.aspx (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1033)')))


dict_keys([])

In [ ]:
# Get dividend history
dividends = getDividend('Attijariwafa')
pd.DataFrame(dividends)

## 6. Index Information

Get market index summaries, weights, and session recaps.

In [ ]:
# Get all index summaries
index_data = getIndex()
print("Available keys:", list(index_data.keys()))
index_data.get('Resume indice', {}).get('MASI', {})

In [ ]:
# Get index weights (pondération)
weights = cas.getPond()
pd.DataFrame(weights).head(10)

In [ ]:
# Get session recap
recap = cas.getIndexRecap()
print("Available keys:", list(recap.keys()) if isinstance(recap, dict) else type(recap))
recap